In [1]:
# import

import os
import pandas as pd
import numpy as np
import random
import json
import glob
from tqdm import tqdm
from pprint import pprint


In [2]:
# test load quick draw
apple = np.load('../data/sketch_rnn/apple.npz',
                encoding='latin1', allow_pickle=True)
print(apple.files)
print(apple['train'][0].shape)
print(apple['train'][0])

['test', 'train', 'valid']
(43, 3)
[[-17 -18   0]
 [-19 -14   0]
 [-32  -2   0]
 [-13   5   0]
 [-17  11   0]
 [ -9  11   0]
 [ -9  48   0]
 [  3  32   0]
 [  9  15   0]
 [ 64  70   0]
 [  9  45   0]
 [ 10  10   0]
 [ 19   3   0]
 [ 12 -15   0]
 [ 22   0   0]
 [ 10   6   0]
 [  5  -2   0]
 [ 15 -34   0]
 [ 37 -37   0]
 [  5 -10   0]
 [  6 -15   0]
 [  6 -33   0]
 [  2 -37   0]
 [ -5 -13   0]
 [-12 -15   0]
 [-48 -33   0]
 [-13  -1   0]
 [-14   6   0]
 [-18  15   0]
 [ -5   8   1]
 [  3 -19   0]
 [ -1 -16   0]
 [ 27 -54   1]
 [-18  28   0]
 [  4  -5   0]
 [ 30  -9   0]
 [ 19  -1   0]
 [ 31  18   0]
 [ -8   0   0]
 [-19   7   0]
 [-25  16   0]
 [-22   1   0]
 [-18  -8   1]]


In [3]:
quickdraw_map = {}
df = pd.read_csv('../outputs/sketchyscene_quickdraw.csv')
df = df.dropna(subset=['quickdraw_label'])
for row in df.itertuples():
    quickdraw = np.load(
        f'../data/sketch_rnn/{row.quickdraw_label}.npz', encoding='latin1', allow_pickle=True)
    quickdraw_map[row.sketchyscene_label] = [row.quickdraw_label, {
        "train": quickdraw["train"],
        "valid": quickdraw["valid"],
        "test": quickdraw["test"],
    }]


In [4]:
len(apple['train'])

70000

In [5]:
def px2int(px: str) -> int:
    return int(px.replace('px', ''))


random.seed(100)
data_types = [("train", "train"), ("val", "valid"), ("test", "test")]
os.makedirs('../data/isketcher', exist_ok=True)
SKETCHY_SCENE_WIDTH = 750
for data_type in data_types:
    data_list = []
    print(f"Now Generating: {data_type[1]}")
    for filename in tqdm(glob.glob(f'../data/sketchy_scene/records/{data_type[0]}/skenew/results1/*.json')):
        with open(filename) as f:
            scene = json.load(f)
        draws = []
        temp = quickdraw_map['apple'][1]["train"]
        for sketch in scene:
            if sketch['category'] in quickdraw_map:
                w = px2int(sketch['width'])
                h = px2int(sketch['height'])
                x = px2int(sketch['left']) + (w // 2)
                y = px2int(sketch['top']) + (h // 2)
                draw = {
                    'label': quickdraw_map[sketch['category']][0],
                    'sketch': random.choice(quickdraw_map[sketch['category']][1][data_type[1]]).tolist(),
                    'position': [x, y, w, h],
                }
                draws.append(draw)
        data_list.append(draws)

    with open(f'../data/isketcher/{data_type[1]}.json', 'w') as f:
        json.dump(data_list, f)


Now Generating: train


100%|██████████| 5617/5617 [00:02<00:00, 1894.12it/s]


Now Generating: valid


100%|██████████| 535/535 [00:00<00:00, 1753.94it/s]


Now Generating: test


100%|██████████| 1113/1113 [00:00<00:00, 1462.34it/s]


In [6]:
with open('../data/isketcher/test.json', 'r') as f:
    test = json.load(f)
print(len(test), len(test[0]), len(test[0][0]['sketch']))
pprint(test[0])

1113 16 64
[{'label': 'cloud',
  'position': [80, 55, 160, 90],
  'sketch': [[-10, -18, 0],
             [-4, -32, 0],
             [4, -16, 0],
             [11, -13, 0],
             [17, -11, 0],
             [16, -8, 0],
             [72, -17, 0],
             [10, -6, 0],
             [30, -7, 0],
             [45, -1, 0],
             [24, 6, 0],
             [5, 5, 0],
             [2, 10, 0],
             [-18, 17, 0],
             [10, 6, 0],
             [30, 8, 0],
             [31, 13, 0],
             [11, 7, 0],
             [7, 9, 0],
             [2, 18, 0],
             [-7, 11, 0],
             [-40, 28, 0],
             [0, 6, 0],
             [41, 17, 0],
             [25, 22, 0],
             [0, 19, 0],
             [-18, 27, 0],
             [-18, 13, 0],
             [-19, 8, 0],
             [-52, 14, 0],
             [5, 0, 0],
             [4, 10, 0],
             [-22, 17, 0],
             [-21, 11, 0],
             [-66, 16, 0],
             [-54, 1, 0],
  